SALES TREND

In [ ]:
#Revenue, Cost, Profit, and Order Trend
SELECT 
    FORMAT_TIMESTAMP('%Y', o.created_at) AS year, -- Year format
    FORMAT_TIMESTAMP('%Y-%m', o.created_at) AS year_month, -- Year and month format
    FORMAT_TIMESTAMP('%Y-%m-%d', o.created_at) AS year_month_day, -- Year, month, and day format
    SUM(oi.sale_price) AS total_revenue,
    SUM(p.cost) AS total_cost,
    SUM(oi.sale_price - p.cost) AS total_profit,
	COUNT(DISTINCT o.order_id) AS total_orders,
    ROUND(AVG(oi.sale_price - p.cost), 2) AS avg_profit_per_order
FROM 
    `bigquery-public-data.thelook_ecommerce.orders` o
JOIN 
    `bigquery-public-data.thelook_ecommerce.order_items` oi
    ON o.order_id = oi.order_id
JOIN
    `bigquery-public-data.thelook_ecommerce.products` p
    ON oi.product_id = p.id
WHERE 
    o.status = 'Complete' -- Only consider completed orders
GROUP BY 
    year, year_month, year_month_day
ORDER BY 
    year_month_day ASC;

CUSTOMER PROFILE

In [ ]:
#AgeofCustomer
SELECT
    CASE 
        WHEN age BETWEEN 0 AND 12 THEN 'Children'
        WHEN age BETWEEN 13 AND 19 THEN 'Teenagers'
        WHEN age BETWEEN 20 AND 39 THEN 'Young Adults'
        WHEN age BETWEEN 40 AND 59 THEN 'Middle-aged'
        WHEN age >= 60 THEN 'Elderly'
        ELSE 'Unknown'
    END AS Age_Group,
    COUNT(*) AS Total_User
FROM `bigquery-public-data.thelook_ecommerce.users`
GROUP BY 
    CASE 
        WHEN age BETWEEN 0 AND 12 THEN 'Children'
        WHEN age BETWEEN 13 AND 19 THEN 'Teenagers'
        WHEN age BETWEEN 20 AND 39 THEN 'Young Adults'
        WHEN age BETWEEN 40 AND 59 THEN 'Middle-aged'
        WHEN age >= 60 THEN 'Elderly'
        ELSE 'Unknown'
    END
ORDER BY
    Total_User DESC;

#GenderofCustomer
SELECT
  gender,
  COUNT(*)
FROM `bigquery-public-data.thelook_ecommerce.users`
GROUP BY
  gender
ORDER BY
  gender DESC;

#TotalUser/Country
SELECT
  country,
  COUNT(*) AS Total_User
FROM `bigquery-public-data.thelook_ecommerce.users`
GROUP BY
  country
ORDER BY
  Total_User DESC;

#GrowthofUser
SELECT 
    EXTRACT(YEAR FROM created_at) AS year,
    COUNT(*) AS Total_User,
FROM 
    `bigquery-public-data.thelook_ecommerce.users`
GROUP BY 
    year
ORDER BY 
   Total_User ASC;

Event by traffic

In [ ]:
#Total visitor by traffic source
SELECT  
    traffic_source, 
	DATE(created_at) AS event_date,
    COUNT(*) AS total_visitors_TS
FROM 
    `bigquery-public-data.thelook_ecommerce.events`
GROUP BY 
    traffic_source, event_date 
ORDER BY 
    total_visitors_TS DESC;

# Total visitor by event type
SELECT  
    event_type,
	DATE(created_at) AS event_date,
    COUNT(*) AS total_visitors_ET
FROM 
    `bigquery-public-data.thelook_ecommerce.events`
GROUP BY 
    event_type, event_date
ORDER BY 
   total_visitors_ET  DESC;

# Total visitor by state
SELECT  
    state,
	DATE(created_at) AS event_date, 
    COUNT(*) AS total_visitors_state
FROM 
    `bigquery-public-data.thelook_ecommerce.events`
GROUP BY 
    state, event_date
ORDER BY 
   total_visitors_state  DESC;

# time series 
SELECT 
    DATE(created_at) AS event_date,
	EXTRACT(DAYOFWEEK FROM created_at) AS day_of_week,
    EXTRACT(HOUR FROM created_at) AS hour_of_day,
    COUNT(*) AS total_visitors
FROM 
    `bigquery-public-data.thelook_ecommerce.eventtype`
GROUP BY 
    day_of_week, hour_of_day, event_date
ORDER BY 
    total_visitors DESC;

In [ ]:
Revenue

In [ ]:
SELECT 
    u.traffic_source,
	DATE(o.created_at) AS order_date,
    ROUND(SUM(p.retail_price), 2) AS total_revenue
FROM 
    `bigquery-public-data.thelook_ecommerce.users` u
JOIN 
    `bigquery-public-data.thelook_ecommerce.orders` o
    ON u.id = o.user_id
JOIN 
    `bigquery-public-data.thelook_ecommerce.order_items` oi
    ON o.order_id = oi.order_id
JOIN 
    `bigquery-public-data.thelook_ecommerce.products` p
    ON oi.product_id = p.id
WHERE 
    o.status = 'Complete'
GROUP BY 
    u.traffic_source, order_date
ORDER BY 
    total_revenue DESC;

SELECT 
    u.city,
    u.state,
	DATE(oi.created_at) AS order_date,
    COUNT(o.order_id) AS total_orders,
    ROUND(SUM(p.retail_price), 2) AS total_revenue
FROM 
    `bigquery-public-data.thelook_ecommerce.orders` o
JOIN 
    `bigquery-public-data.thelook_ecommerce.users` u
    ON o.user_id = u.id
JOIN 
    `bigquery-public-data.thelook_ecommerce.order_items` oi
    ON o.order_id = oi.order_id
JOIN 
    `bigquery-public-data.thelook_ecommerce.products` p
    ON oi.product_id = p.id
WHERE 
    o.status = 'Complete'
GROUP BY 
    u.city, u.state, order_date
ORDER BY 
    total_revenue DESC;

SELECT 
    product_id,
    p.name AS product_name,
	DATE(oi.created_at) AS order_date,
    COUNT(order_id) AS total_reorders,
    ROUND(SUM(p.retail_price), 2) AS total_revenue
FROM 
    `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN 
    `bigquery-public-data.thelook_ecommerce.products` p
    ON oi.product_id = p.id
GROUP BY 
    product_id, product_name, order_date
ORDER BY 
    total_reorders DESC;

SELECT 
    p.category AS product_category,
	DATE(oi.created_at) AS order_date,	
    COUNT(oi.product_id) AS total_sold,
    ROUND(SUM(p.retail_price), 2) AS total_revenue
FROM 
    `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN 
    `bigquery-public-data.thelook_ecommerce.products` p
    ON oi.product_id = p.id
GROUP BY 
    p.category, order_date
ORDER BY 
    total_sold DESC;

SELECT 
    p.brand AS product_brand,
	DATE(oi.created_at) AS order_date,	
    COUNT(oi.product_id) AS total_sold,
    ROUND(SUM(p.retail_price), 2) AS total_revenue
FROM 
    `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN 
    `bigquery-public-data.thelook_ecommerce.products` p
    ON oi.product_id = p.id
GROUP BY 
    p.brand, order_date
ORDER BY 
    total_sold DESC;

In [ ]:
Transaction Behavior

In [ ]:
#Average items per order
SELECT 
    ROUND(AVG(num_of_item),0) AS avg_items_per_order 
FROM 
    `bigquery-public-data.thelook_ecommerce.orders`;

#Average spending per transaction
SELECT 
    o.order_id,
	DATE(o.created_at) AS order_date,
    ROUND(SUM(p.retail_price) / COUNT(DISTINCT o.order_id), 2) AS avg_spending_per_transaction
FROM 
    `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN 
    `bigquery-public-data.thelook_ecommerce.products` p 
    ON oi.product_id = p.id
JOIN 
    `bigquery-public-data.thelook_ecommerce.orders` o
    ON oi.order_id = o.order_id
GROUP BY 
    o.order_id, order_date
ORDER BY 
    avg_spending_per_transaction DESC;

#Average days between orders
WITH order_differences AS (
    SELECT 
        user_id,
  		DATE(created_at) AS order_date,
        DATE_DIFF(created_at, LAG(created_at) OVER (PARTITION BY user_id ORDER BY created_at ASC), DAY) AS days_between_orders
    FROM 
        `bigquery-public-data.thelook_ecommerce.orders`
    WHERE 
        status = 'Complete'
)
SELECT 
    user_id,
    ROUND(AVG(days_between_orders), 2) AS avg_days_between_orders
FROM 
    order_differences
WHERE 
    days_between_orders IS NOT NULL
GROUP BY 
    user_id, order_date
ORDER BY 
    avg_days_between_orders;

#Total repeat customer
SELECT 
    COUNT(user_id) AS repeat_customers 
FROM 
    (SELECT 
        user_id, 
        COUNT(order_id) AS order_count 
     FROM 
        `bigquery-public-data.thelook_ecommerce.orders` 
     GROUP BY 
        user_id 
     HAVING 
        order_count > 1);


#Most selling product
SELECT 
    product_id,
    p.name AS product_name,
	DATE(oi.created_at) AS order_date,
    COUNT(order_id) AS total_reorders
FROM 
    `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN 
    `bigquery-public-data.thelook_ecommerce.products` p
    ON oi.product_id = p.id
GROUP BY 
    product_id, product_name, order_date
ORDER BY 
    total_reorders DESC;

#Peak hour to purchase
SELECT
    DATE(created_at) AS event_date,
    EXTRACT(DAYOFWEEK FROM created_at) AS day_of_week,
    EXTRACT(HOUR FROM created_at) AS hour_of_day,
    COUNT(*) AS total_orders,
FROM
    `bigquery-public-data.thelook_ecommerce.events`
WHERE
    event_type = "purchase"
GROUP BY
    day_of_week, hour_of_day, event_date
ORDER BY
    total_orders DESC

In [ ]:
Loyalty & Churn Customer

In [ ]:
SELECT 
    u.id AS user_id,
	DATE(oi.created_at) AS order_date,
    CONCAT(u.first_name, ' ', u.last_name) AS customer_name,
    ROUND(SUM(p.retail_price), 2) AS total_spent
FROM 
    `bigquery-public-data.thelook_ecommerce.orders` o
JOIN 
    `bigquery-public-data.thelook_ecommerce.users` u
    ON o.user_id = u.id
JOIN 
    `bigquery-public-data.thelook_ecommerce.order_items` oi
    ON o.order_id = oi.order_id
JOIN 
    `bigquery-public-data.thelook_ecommerce.products` p
    ON oi.product_id = p.id
WHERE 
    o.status = 'Complete'
GROUP BY 
    u.id, customer_name, order_date
ORDER BY 
    total_spent DESC;

SELECT 
    u.id AS user_id,
	DATE(o.created_at) AS order_date,
    CONCAT(u.first_name, ' ', u.last_name) AS customer_name,
    p.category AS product_category,
    COUNT(o.order_id) AS total_returns
FROM 
    `bigquery-public-data.thelook_ecommerce.orders` o
JOIN 
    `bigquery-public-data.thelook_ecommerce.users` u
    ON o.user_id = u.id
JOIN 
    `bigquery-public-data.thelook_ecommerce.order_items` oi
    ON o.order_id = oi.order_id
JOIN 
    `bigquery-public-data.thelook_ecommerce.products` p
    ON oi.product_id = p.id
WHERE 
    o.returned_at IS NOT NULL
GROUP BY 
    user_id, customer_name, p.category, order_date
ORDER BY 
    total_returns DESC
LIMIT 10;

SELECT 
    u.id AS user_id,
	DATE(o.created_at) AS order_date,
    CONCAT(u.first_name, ' ', u.last_name) AS customer_name,
    MAX(o.created_at) AS last_purchase_date
FROM 
    `bigquery-public-data.thelook_ecommerce.users` u
LEFT JOIN 
    `bigquery-public-data.thelook_ecommerce.orders` o
    ON u.id = o.user_id
GROUP BY 
    u.id, customer_name, order_date
HAVING    
    TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), MAX(o.created_at), DAY) > 180
ORDER BY 
    last_purchase_date DESC;

SELECT 
    e.user_id,
	DATE(o.created_at) AS order_date,
    COUNT(e.id) AS total_events,
    COUNT(o.order_id) AS total_orders
FROM 
    `bigquery-public-data.thelook_ecommerce.events` e
LEFT JOIN 
    `bigquery-public-data.thelook_ecommerce.orders` o
    ON e.user_id = o.user_id
GROUP BY 
    e.user_id, order_date
HAVING 
    total_orders = 0 AND total_events > 10
ORDER BY 
    total_events DESC;